In [ ]:
{-# LANGUAGE BangPatterns, ScopedTypeVariables #-}
import Control.Monad
import Control.Monad.Primitive

import Numeric.SpecFunctions
import Numeric.MathFunctions.Constants
import Numeric.MathFunctions.Comparison

import Text.Printf(printf)

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy hiding (within)

import Debug.Trace

:l NB/Plot

# Incomplete beta

Quick reminder about beta function and (regularized) incomplete beta functions:

Beta function:
$$B(a,b) = \int_0^1 t^{a-1}(1 - t)^{b-1} \,dt $$

Incomplete beta:
$$B(x; a,b) = \int_0^x t^{a-1}(1 - t)^{b-1} \,dt \qquad x \in [0,1]$$

Regularized incomplete beta (from now it'll be referred to simply as incomplete beta)
$$I(x; a,b) = \frac{B(x; a,b)}{B(a,b)}$$


# Bug itself

Was found in 0.1.7.0 and fixed in 0.1.8.0

Following plot illustrates bug perfectly. It's plot of incomplete beta for very small range (200 ULPs)

In [ ]:
toRenderable $
  ulpPlot [incompleteBeta 0.5 0.5] 1e-10 200

Let copy sourse code here yet again and fix bug along the way. Way how bug was found is described below.

In [ ]:
-- | Regularized incomplete beta function. Uses algorithm AS63 by
-- Majumder and Bhattachrjee and quadrature approximation for large
-- /p/ and /q/.
incompleteBeta' :: Double -- ^ /p/ > 0
               -> Double -- ^ /q/ > 0
               -> Double -- ^ /x/, must lie in [0,1] range
               -> Double
incompleteBeta' p q = incompleteBeta_ b p q
  where
    b = logGammaL p + logGammaL q - logGammaL (p + q)

-- | Regularized incomplete beta function. Same as 'incompleteBeta'
-- but also takes logarithm of beta function as parameter.
incompleteBeta_ :: Double -- ^ logarithm of beta function for given /p/ and /q/
                -> Double -- ^ /p/ > 0
                -> Double -- ^ /q/ > 0
                -> Double -- ^ /x/, must lie in [0,1] range
                -> Double
incompleteBeta_ beta p q x
  | p <= 0 || q <= 0            =
      error $ printf "incompleteBeta_: p <= 0 || q <= 0. p=%g q=%g x=%g" p q x
  | x <  0 || x >  1 || isNaN x =
      error $ printf "incompletBeta_: x out of [0,1] range. p=%g q=%g x=%g" p q x
  | x == 0 || x == 1            = x
  | p >= (p+q) * x   = snd $ incompleteBetaWorker beta p q x
  | otherwise        = error "Missing case" -- 1 - incompleteBetaWorker beta q p (1 - x)


-- Worker for incomplete beta function. It is separate function to
-- avoid confusion with parameter during parameter swapping
incompleteBetaWorker :: Double -> Double -> Double -> Double -> ([Double],Double)
incompleteBetaWorker beta p q x
  -- For very large p and q this method becomes very slow so another
  -- method is used.
  | p > 3000 && q > 3000 = error "removed"
  | otherwise            = loop (p+q) (truncate $ q + cx * (p+q)) 1 1 1 [1]
  where
    -- Constants
    eps = 2e-16    
    cx  = 1 - x
    -- Loop
    loop !psq (ns :: Int) ai term betain terms
      | done      =
          ( reverse $ term' : terms
          , betain' * x ** p * cx ** (q - 1) / p / exp (logBeta p q)
          )
          -- term' : terms
      | otherwise = loop psq' (ns - 1) (ai + 1) term' betain' (term':terms)
      where
        -- New values
        term'   = term * fact / (p + ai)
        betain' = betain + term'
        fact | ns >  0   = (q - ai) * x/cx
             | ns == 0   = (q - ai) * x
             | otherwise = psq * x
        -- Iterations are complete
        done = db <= eps && db <= eps*betain' where db = abs term'
        psq' = if ns < 0 then psq + 1 else psq

It's ietarative expansion. So first idea is to check whether convergence criterion is too lax. Redusing `eps` in `incompleteBetaWorker` definition have to effect. Maybe normaliztion factor `exp( p * log x + (q - 1) * log cx - beta) / p` is to blame? Let plot it

In [ ]:
toRenderable $
  let p = 0.5
      q = 0.5
      beta = logBeta p q
  in ulpPlot [\x -> exp (p * log x + (q - 1) * log (1-x) - beta)] 1e-10 200

Very familiar ladder! It seems we on the right way. Maybe we could rewrite this expression in more precise way...

In [ ]:
toRenderable $
  let p = 0.5
      q = 0.5
      beta = logBeta p q
  in ulpPlot [ \x -> exp (p * log x + (q - 1) * log (1-x) - beta)
             , \x -> x**p * (1-x)**(q-1) / exp beta
             ] 1e-10 200

Looks like this is fix for bug now we should make sure that this fix does not break things in other places.

# exp (log x * a)

This is very common idiom for computing x\*\*a but do we lose presition there? To measure it we need to determine into how many ULPS `log` transforms `ULP` interval. So for arbitrary $f$:

$$f(x[1+\varepsilon]) = f(x) + f'(x)x\varepsilon = f(x)\left[ 1 + \frac{f'(x)x}{f(x)}\varepsilon\right]$$

So 1 ULP is transformed into $x f'(x) / f(x)$. 
If this quantity is less than 1 we lose information because several inputs bound to be transformed into same output. Now let look just how bad things for `log`

In [ ]:
toRenderable 
  $ plotFunctions [\x ->  min 5 $ abs $ 1 / log x, const 1] (1e-6, 10)

So things are not good. Let plot $x^a$ calculated using `log` and using `**` for $x \ll 1$ and $x \gg 1$ and different `a`

In [ ]:
toRenderable $
  let a  = 0.2
      x0 = 1e-10
  in ulpPlot [\x -> exp (log x * a), (**a)] x0 200
  
toRenderable $
  let a  = 1
      x0 = 1e-10
  in ulpPlot [\x -> exp (log x * a), (**a)] x0 200
  
toRenderable $
  let a  = 3
      x0 = 1e-10
  in ulpPlot [\x -> exp (log x * a), (**a)] x0 200

In [ ]:
toRenderable $
  let a  = 0.2
      x0 = 1e3
  in ulpPlot [\x -> exp (log x * a), (**a)] x0 200
  
toRenderable $
  let a  = 1
      x0 = 1e3
  in ulpPlot [\x -> exp (log x * a), (**a)] x0 200
  
toRenderable $
  let a  = 3
      x0 = 1e3
  in ulpPlot [\x -> exp (log x * a), (**a)] x0 200